In [4]:
from bs4 import BeautifulSoup
import requests


def get_faqs():
    response = requests.get('https://www.swinburneonline.edu.au/faqs/')
    soup = BeautifulSoup(response.content, 'html.parser')
    # get faq cards
    faqs_cards = soup.select('.faqs-group .card')
    result = []
    # loop through and get questions and answers
    for faq in faqs_cards:
        question_el = faq.select_one('.card-header h5 > div:nth-child(2)')
        answer_el = faq.select_one('.card-body .content')
        # add to result if question and answer exist
        if question_el and answer_el:
            question = question_el.get_text(strip=True)
            answer = answer_el.get_text(strip=True)
            result.append((question, answer))
    return result

[('What support can I expect?', 'As a Swinburne Online student, you’ll have support for extended hours, seven days a week, with Student Advisors available to help with anything from tech support to research advice and dedicated online tutors in each of your units. Learn more aboutyour support.'), ('Is an MBA too advanced for someone who has never studied?', 'Yes, Swinburne Online’s MBA is a postgraduate degree, meaning that previous tertiary study is an entry requirement for the course. If you have not studied at a tertiary level before, you may seek entry into the course via theGraduate Certificate of Business Administration. See the entry requirements for more information.'), ('As I am studying online, can I pursue full-time opportunities throughout my studies?', 'Yes, there are no requisite live or in-person sessions, so you can elect to study anytime, anywhere. If you find the combination of full-time work and full-time study to be too much, you can choose to study part-time.'), ('

In [6]:
from transformers import BertTokenizer, TFBertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
input_data = []
output_data = []

max_length = 256  # Set the maximum length for input/output sequences

# Loop through each FAQ entry
for faq in get_faqs():
    question = faq[0]
    answer = faq[1]
#     Print the values for testing purposes
    print('Question:', question)
    print('Answer:', answer)
#
#     # Tokenize the question and answer
#     question_tokens = tokenizer.encode(question, add_special_tokens=False)
#     answer_tokens = tokenizer.encode(answer, add_special_tokens=False)
#
#     # Convert tokens to input/output pairs
#     input_tokens = question_tokens + [tokenizer.sep_token_id] + answer_tokens
#     output_tokens = answer_tokens + [tokenizer.sep_token_id]
#
#     # Pad or truncate the sequences to a fixed length
#     if len(input_tokens) > max_length:
#         input_tokens = input_tokens[:max_length]
#     else:
#         input_tokens = np.pad(input_tokens, (0, max_length - len(input_tokens)), mode='constant', constant_values=0)
#
#     if len(output_tokens) > max_length:
#         output_tokens = output_tokens[:max_length]
#     else:
#         output_tokens = np.pad(output_tokens, (0, max_length - len(output_tokens)), mode='constant', constant_values=0)
#
#     # Append to input/output data
#     input_data.append(input_tokens)
#     output_data.append(output_tokens)
#
# # Convert input/output data to numpy arrays for further processing
# input_data = np.array(input_data)
# output_data = np.array(output_data)
#
#
#
# # FINE-TUNING THE MODEL
# class MyFineTuningModel(tf.keras.Model):
#     def __init__(self, bert_model):
#         super(MyFineTuningModel, self).__init__()
#         self.bert = bert_model
#         self.dense = tf.keras.layers.Dense(max_length, activation='softmax')
#
#     def call(self, inputs):
#         bert_output = self.bert(inputs)[0]
#         dense_output = self.dense(bert_output)
#         return dense_output
#
# # Load the pre-trained BERT model
# bert_model = TFBertModel.from_pretrained('bert-base-uncased')
#
# # Create an instance of the fine-tuning model
# my_model = MyFineTuningModel(bert_model)
#
# # Set up the optimizer and loss function
# optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
# loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
#
# # Compile the model
# my_model.compile(optimizer=optimizer, loss=loss_function)
#
# # Train the model
# my_model.fit(input_data, output_data, epochs=5, batch_size=32)
#
# # Generate predictions
# predictions = my_model.predict(input_data)
#
# # Postprocess the predictions
# predicted_answers = []
# for prediction in predictions:
#     predicted_answer_tokens = np.argmax(prediction, axis=-1)
#     predicted_answer = tokenizer.decode(predicted_answer_tokens)
#     predicted_answers.append(predicted_answer)
#
# # Print the predicted answers
# for i in range(len(faq_list)):
#     print('Question:', faq_list[i]['question'])
#     print('Predicted Answer:', predicted_answers[i])
#     print('Actual Answer:', faq_list[i]['answer'])
#     print('---')


Question: What support can I expect?
Answer: As a Swinburne Online student, you’ll have support for extended hours, seven days a week, with Student Advisors available to help with anything from tech support to research advice and dedicated online tutors in each of your units. Learn more aboutyour support.
Question: Is an MBA too advanced for someone who has never studied?
Answer: Yes, Swinburne Online’s MBA is a postgraduate degree, meaning that previous tertiary study is an entry requirement for the course. If you have not studied at a tertiary level before, you may seek entry into the course via theGraduate Certificate of Business Administration. See the entry requirements for more information.
Question: As I am studying online, can I pursue full-time opportunities throughout my studies?
Answer: Yes, there are no requisite live or in-person sessions, so you can elect to study anytime, anywhere. If you find the combination of full-time work and full-time study to be too much, you can 